In [101]:

import sqlite3
import pandas as pd

pd.set_option('display.max_rows', None)

conn = sqlite3.Connection('../backend/data.db')
cur = conn.cursor()

# pd.read_sql("""
#     select article_id, title 
#     from articles 
#     where count > 20
#     order by random()
#     limit 15
# """, conn)

In [1]:

import sqlite3
import pandas as pd

pd.set_option('display.max_rows', None)

conn = sqlite3.Connection('../backend/data.db')
cur = conn.cursor()
# df = pd.read_csv('data/scrape_articles_top5_len40_stride15.csv')
# df = df[~df.clean_title.isna()]
# df[df.clean_title.str.contains('tom c')]

In [1]:

import sqlite3
import pandas as pd

pd.set_option('display.max_rows', None)

conn = sqlite3.Connection('../backend/data.db')
cur = conn.cursor()
# pd.read_sql("""
#     select article_id, count(*)
#     from articles
#     group by article_id
#     order by count(*) desc
#     limit 100
# """, conn)

pd.read_sql("""
    select *
    from chunks
    limit 1
""", conn).iloc[0]['chunk']

# cur.execute("""
# DELETE FROM articles
# WHERE rowid NOT IN (
#   SELECT MIN(rowid)
#   FROM articles
#   GROUP BY article_id
# );""")

# cur.execute("""
# DELETE FROM chunks
# WHERE rowid NOT IN (
#   SELECT MIN(rowid)
#   FROM chunks
#   GROUP BY chunk_id
# );""")



'Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the'

In [3]:
pd.read_sql("""
    select count(*)
    from chunks
""", conn)

,count(*)
0,52537512


In [ ]:
# distinct article_id 1003432
# distinct chunk_id 82537812

In [7]:
for key, value in selected.items():
    if key.lower().__contains__('tom c'):
        print(key, value)

Tom Conti 29
Tom C. Korologos 1
Tom Clancy 137
Tom Cohen 1
Custom Coaches 2
Alstom Citadis 25
Tom Colicchio 3
Tom Cleverley 4
Tom Coughlin 24
Tom Cora 8
Phantom Corsair 2
Tom Cotton 34
Amiga custom chips 14
Tom Candiotti 8
Tom Collins 8
Tom Coronel 4
Tom Cruise 321
Tom Cochrane 24
Tom C. Clark 39
Atom cluster 2
Tom Connolly (umpire) 2
Tom Curry (rugby union) 3
Tom Cribb 11
Alstom Coradia 5
Tom Clancy's Rainbow Six Siege 18
Tom Crean (basketball) 2
Stompin' Tom Connors 22
Tom Cheney (cartoonist) 2
Black Tom Cassidy 11
Tom Carper 29
Tom Campbell Black 7
Tom Clancy's Splinter Cell Double Agent 1
Tom Clarke (politician) 5
Alstom Coradia LINT 6
Tom Clark (poet) 4
Tom Tom Club 37
Tom Clarke (Irish republican) 26
Tom Courtenay 56
Tom Cavalcante 1
Tom Cairney 3
Tom Crean (explorer) 11
Tom Chavez 1
Tom Constanten 18
Tom Cooper (driver) 2
Tom Chaplin 9
Tom Cartwright 5
Tom Cole (racing driver) 2
Tom Chambers (basketball) 14
Tom Coughlin Jay Fund Foundation 2
Tom Cooper (footballer) 3
Tom Coyne (

In [1]:
with open("data/links.txt", "r", encoding="utf-8", errors="replace") as f:
    lines = [line.rstrip("\n") for line in f]

In [4]:
[x for x in lines if x.lower().__contains__('tom cr')][: 10]

['Tom Cruise',
 'Tom Cruise',
 'Tom Cribb',
 'Tom Crean (basketball)|Tom Crean',
 'Tom Cruise',
 'Tom Cruise',
 'Tom Cruise',
 'Tom Cruise',
 'Tom Cruise',
 'Tom Cruise']

In [13]:
import sqlite3
import pandas as pd

pd.set_option('display.max_rows', None)

conn = sqlite3.Connection('../backend/data.db')

pd.read_sql("""
    select *
    from articles
    where title like 'planet%'
    order by title desc
""", conn)

,article_id,title,clean_title,url,count
0,87916,Planets in science fiction,planets in science fiction,https://en.wikipedia.org/wiki/Planets%20in%20s...,14
1,30872816,Planets in astrology,planets in astrology,https://en.wikipedia.org/wiki/Planets%20in%20a...,20
2,192500,Planetesimal,planetesimal,https://en.wikipedia.org/wiki/Planetesimal,47
3,564990,Planetes,planetes,https://en.wikipedia.org/wiki/Planetes,9
4,26903467,Planetary-mass object,planetary-mass object,https://en.wikipedia.org/wiki/Planetary-mass%2...,9
5,27361858,Planetary-mass moon,planetary-mass moon,https://en.wikipedia.org/wiki/Planetary-mass%2...,11
6,252372,Planetary system,planetary system,https://en.wikipedia.org/wiki/Planetary%20system,66
7,34661457,Planetary surface,planetary surface,https://en.wikipedia.org/wiki/Planetary%20surface,12
8,36224143,Planetary science,planetary science,https://en.wikipedia.org/wiki/Planetary%20science,47
9,1162325,Planetary romance,planetary romance,https://en.wikipedia.org/wiki/Planetary%20romance,13


In [6]:
import json

js = []
with open('../backend/ai_targets.jsonl', 'r') as file:
    for i, line in enumerate(file):
        if line.strip() == '':
            continue
        print(line.strip())
        j = json.loads(line.strip())
        j['day'] = i + 1
        del(j['count'])
        js.append(j)

with open('../backend/targets.jsonl', 'w', encoding='utf-8') as file:
    for j in js:
        file.write(json.dumps(j) + '\n')

{"article_id": 18717883, "count": -1}
{"article_id": 178690, "count": -1}
{"article_id": 36536119, "count": -1}
{"article_id": 991, "count": -1}
{"article_id": 39626432, "count": -1}
{"article_id": 463734, "count": -1}
{"article_id": 42898, "count": -1}
{"article_id": 4068, "count": -1}
{"article_id": 191280, "count": -1}
{"article_id": 163115, "count": -1}
{"article_id": 44940, "count": -1}
{"article_id": 87219, "count": -1}
{"article_id": 3254510, "count": -1}
{"article_id": 62092, "count": -1}
{"article_id": 371248, "count": -1}
{"article_id": 1776, "count": -1}
{"article_id": 210560, "count": -1}
{"article_id": 265554, "count": -1}
{"article_id": 23545010, "count": -1}
{"article_id": 52135, "count": -1}
{"article_id": 48146, "count": -1}
{"article_id": 158548, "count": -1}
{"article_id": 2541904, "count": -1}


In [144]:
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI
import os 
import pandas as pd
load_dotenv()

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
DB_PATH = '../backend/data.db'

conn = sqlite3.Connection(DB_PATH)
cur = conn.cursor()
# cur.execute("""
#     select chunk
#     from chunks
#     where article_id == ?
# """, (19017269,))
# chunks = ' '.join([x[0] for x in cur.fetchall()]).split()

pd.read_sql("""
    select *
    from articles 
    where count > 60
    order by random()
    limit 10
""", conn)

# pd.read_sql("select * from articles where title like '%seinfeld%' order by count desc""", conn)

# cur.fetchall()

,article_id,title,clean_title,url,count
0,408362,American Music Awards,american music awards,https://en.wikipedia.org/wiki/American%20Music...,146
1,4721,British Empire,british empire,https://en.wikipedia.org/wiki/British%20Empire,1920
2,64485,New Spain,new spain,https://en.wikipedia.org/wiki/New%20Spain,518
3,50556,James IV of Scotland,james iv of scotland,https://en.wikipedia.org/wiki/James%20IV%20of%...,148
4,72297,Lebensraum,lebensraum,https://en.wikipedia.org/wiki/Lebensraum,97
5,129195,Alnwick,alnwick,https://en.wikipedia.org/wiki/Alnwick,61
6,1181466,Treccani,treccani,https://en.wikipedia.org/wiki/Treccani,64
7,1007676,The Dean Martin Show,the dean martin show,https://en.wikipedia.org/wiki/The%20Dean%20Mar...,68
8,310824,Marshal of France,marshal of france,https://en.wikipedia.org/wiki/Marshal%20of%20F...,242
9,180466,EasyJet,easyjet,https://en.wikipedia.org/wiki/EasyJet,137


In [ ]:
x = """
"""

ids = []
for line in x.split('\n'):
    article_id, title = line.strip().split(maxsplit=1)
    article_id = article_id.strip()
    ids.append(article_id)

import random 

while ids:
    id = random.choice(ids)
    ids.remove(id)
    print('{' + f'"article_id": {id}, "count": -1' + '}')

{"article_id": 39626432, "count": -1}
{"article_id": 463734, "count": -1}
{"article_id": 42898, "count": -1}
{"article_id": 4068, "count": -1}
{"article_id": 191280, "count": -1}
{"article_id": 163115, "count": -1}
{"article_id": 44940, "count": -1}
{"article_id": 87219, "count": -1}
{"article_id": 3254510, "count": -1}
{"article_id": 62092, "count": -1}
{"article_id": 371248, "count": -1}
{"article_id": 1776, "count": -1}
{"article_id": 210560, "count": -1}
{"article_id": 265554, "count": -1}
{"article_id": 23545010, "count": -1}
{"article_id": 52135, "count": -1}
{"article_id": 48146, "count": -1}
{"article_id": 158548, "count": -1}
{"article_id": 2541904, "count": -1}
{"article_id": 100627, "count": -1}
{"article_id": 295855, "count": -1}
{"article_id": 5261, "count": -1}
{"article_id": 3969, "count": -1}
{"article_id": 72890, "count": -1}
{"article_id": 47700, "count": -1}
{"article_id": 45772, "count": -1}
{"article_id": 145804, "count": -1}
{"article_id": 980865, "count": -1}
{"a

In [3]:
import sqlite3
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI
import os 

load_dotenv()

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
DB_PATH = 'data/data.db'

def get_daily_word_vector_live(daily_word: int):
    conn = sqlite3.Connection(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        select chunk
        from chunks
        where article_id == ?
    """, (daily_word,))
    chunks = [x[0] for x in cur.fetchall()]
    result = client.embeddings.create(
        input=chunks,
        model="text-embedding-3-small"
    )
    return np.array([np.array(x.embedding) for x in result.data]).mean(axis=0)

In [5]:
get_daily_word_vector_live(71329952).shape

(1536,)

In [15]:
import sqlite3
import numpy as np

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()
# cur.execute("""
#     select a.title, c.chunk
#     from embeddings e
#     join articles a
#         using (article_id)
#     join chunks c
#         using (chunk_id)
#     where e.id == ?
# """, (167680,))
cur.execute("""
    select chunk_id, count(*)
    from chunks
    group by chunk_id
    order by count(*) desc
    limit 5
""")
for row in cur.fetchall():
    print(row)
conn.close()

(0, 2)
(1, 2)
(2, 2)
(3, 2)
(4, 2)


In [1]:
import sqlite3
import numpy as np

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()
cur.execute("""
    select *
    from articles
    where article_id == ?
""", (19017269,))
cur.fetchall()


[(19017269,
  'World population',
  'world population',
  'https://en.wikipedia.org/wiki/World%20population',
  67)]

In [122]:
conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()

article_id = 44579187

cur.execute("""
    select e.vector, e.chunk_id, c.chunk
    from (
        select vector, chunk_id 
        from embeddings
        where article_id == ?
    ) as e
    join (
        select chunk_id, chunk
        from chunks
        where article_id == ?
    ) as c
        using (chunk_id)
""", (article_id, article_id))

guess = cur.fetchall()
guess_matrix = np.array([np.frombuffer(row[0]) for row in guess])

(159563,
 'both old and new anime titles both on home video and theatrically, aided by the Scotland Loves Anime film festival. From 2015 to 2018, Anime Limited served as the distributors for Funimation in the British Isles. History On 14 December 2012, Kazé UK representative and Scotland Loves Anime director Andrew Partridge launched the social media and website for Anime Limited, and stated that he would be launching a new anime distribution company in 2013, with',
 0.7591181392379761)

In [ ]:
numerator = guess_matrix @ target
denominator_rhs = np.sqrt(np.sum(target ** 2)) 
denominator_lhs = np.sqrt(np.sum(guess_matrix ** 2, axis=1))
denominator = denominator_lhs * denominator_rhs
distances = 1 - (numerator / denominator)
indices = np.argsort(distances)

[(guess[i][1], guess[i][2], distances[i]) for i in indices][: limit]

array([0.79301268, 0.75911814, 0.77018558, 0.78948745, 0.80729594,
       0.84301049, 0.84762888, 0.82576959, 0.82698408, 0.82828016,
       0.79397142, 0.77697281])

In [ ]:
import time

times = []

for _ in range(10000):
    start = time.time()
    numerator = guess @ target
    denominator_rhs = np.sqrt(np.sum(target ** 2)) 
    denominator_lhs = np.sqrt(np.sum(guess ** 2, axis=1))
    denominator = denominator_lhs * denominator_rhs
    distance = 1 - (numerator / denominator)
    times.append((time.time() - start) * 10000)
np.mean(times)

0.5156283378601074

In [101]:
from scipy.spatial.distance import cosine

times = []

for _ in range(10000):
    start = time.time()
    brute = []
    for i in range(12):
        brute.append(cosine(target, guess[i, :]))
    brute = np.array(brute)
    times.append((time.time() - start) * 10000)
np.mean(times)

2.6936564445495605

In [ ]:
import pandas as pd

first = True
for chunk in pd.read_csv('data/chunks_high_len.csv', chunksize=10000):
    if first:
        c = chunk.drop('chunk', axis=1)
        first = False
        continue
    c = pd.concat([chunk.drop('chunk', axis=1), c], axis=0)

In [6]:
import os
import pandas as pd 

a = pd.read_parquet('data/articles.parquet')

In [1]:
import pandas as pd
import sqlite3 
import json 

DB_PATH = '../backend/data.db'

conn = sqlite3.Connection(DB_PATH)